## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-21-50-30 +0000,startribune,"Byron Buxton hits for the cycle, driving the T...",https://www.startribune.com/minnesota-twins-by...
1,2025-07-12-21-49-47 +0000,bbc,Gaza hospital says 24 people killed near aid s...,https://www.bbc.com/news/articles/cp90v2ng70yo
2,2025-07-12-21-40-49 +0000,nypost,These are the forgotten NYC nabes that became ...,https://nypost.com/2025/07/12/real-estate/medi...
3,2025-07-12-21-39-55 +0000,bbc,Morocco and Zambia win to reach Wafcon quarter...,https://www.bbc.com/sport/football/articles/cl...
4,2025-07-12-21-34-16 +0000,nypost,21-year-old NJ resident wins Leonardo DiCaprio...,https://nypost.com/2025/07/12/lifestyle/this-2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,18
103,air,6
35,year,5
19,nyc,5
245,india,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...,46
21,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,43
28,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...,39
68,2025-07-11-22-51-48 +0000,nypost,Dow drops nearly 300 points after Trump teases...,https://nypost.com/2025/07/11/us-news/dow-drop...,39
60,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...,38


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,46,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...
21,25,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
4,23,2025-07-12-21-34-16 +0000,nypost,21-year-old NJ resident wins Leonardo DiCaprio...,https://nypost.com/2025/07/12/lifestyle/this-2...
41,23,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo
60,20,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...
22,19,2025-07-12-16-58-40 +0000,nypost,College student attacked over NYC parking spot...,https://nypost.com/2025/07/12/us-news/college-...
11,17,2025-07-12-20-22-44 +0000,nypost,"Masked, gun-toting duo pistol-whip, rip $75K i...",https://nypost.com/2025/07/12/us-news/masked-g...
6,16,2025-07-12-21-11-05 +0000,bbc,The mushroom killer was obsessed with true cri...,https://www.bbc.com/news/articles/c0m8glx2zleo
52,16,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
35,15,2025-07-12-13-13-54 +0000,startribune,Minneapolis rain gardens were built to reduce ...,https://www.startribune.com/minneapolis-rain-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
